# Write MMTF Subset Demo

Simple example writting a subset of mmtf files

## Imports

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader, mmtfWriter
from mmtfPyspark.filters import ExperimentalMethods, Resolution, RFree
from mmtfPyspark.structureViewer import view_structure

#### Configure Spark 

In [2]:
spark = SparkSession.builder.appName("WriteMmtfCustomSubsetDemo").getOrCreate()

## Read in a fractions of entries from a local Hadoop Sequence File

In [3]:
path = "../../resources/mmtf_full_sample/"
fraction= 0.5
seed = 123

pdb = mmtfReader.read_sequence_file(path, fraction = fraction, seed = seed)

count = pdb.count()

print(f'number of pdb entries read : {count}')

number of pdb entries read : 4855


## Retain high resolution X-ray structures

In [4]:
pdb = pdb.filter(ExperimentalMethods(ExperimentalMethods.X_RAY_DIFFRACTION)) \
         .filter(Resolution(0,2.0)) \
         .filter(RFree(0,2.0))

print(f'number of pdb entries left : {pdb.count()}')

number of pdb entries left : 4028


## Visualize Structures

In [5]:
structures = pdb.keys().collect()
view_structure(structures);

interactive(children=(IntSlider(value=0, continuous_update=False, description='Structure', max=4027), Output()…

## Save this subset in a Hadoop Sequence File

In [6]:
write_path = "./mmtf_subset_xray"

# Reduce RDD to 8 partitiions
pdb = pdb.coalesce(8)
mmtfWriter.write_sequence_file(write_path, pdb)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsHadoopFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 37, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/anaconda3/envs/mmtf-pyspark/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/anaconda3/envs/mmtf-pyspark/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/anaconda3/envs/mmtf-pyspark/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/anaconda3/envs/mmtf-pyspark/lib/python3.7/site-packages/pyspark/python

## Terminate Spark

In [ ]:
spark.stop()